# How To Create A New Reference File Using A MIRI Data Model

This first example will show how to create a very simple reference file (GAIN) using the MIRI data models. The MIRI calibration data products are described on this wiki page:

http://miri.ster.kuleuven.be/bin/view/Internal/CalDataProducts

To create a new file from scratch, it is necessary to know which data model corresponds to which reference file. A mapping between reference file and data model can be found on this wiki page:

http://miri.ster.kuleuven.be/bin/view/Internal/Software/MiriNewDataProductsUse


All the reference data models may be found in Python package miri.datamodels.cdp. In the following Python code the GAIN data model is imported.

In [1]:
import numpy as np
from miri.datamodels.cdp import MiriGainModel

It is assumed you have some Python code which derives the MIRI GAIN calibration and stores it in a numpy array.
In this example, a 3x3 array is created with dummy data and the array is used to create a new GAIN data model. 

In [2]:
data3x3 = np.array([[1.0,1.2,1.1],[1.3,1.2,1.0],[1.1,0.8,0.9]])
gain_model = MiriGainModel( data=data3x3 )

After the data model has been created, you can query the contents of the data model metadata by displaying the contents of the "info" attribute:

In [3]:
print( gain_model.info )

MiriGainModel: 
Metadata
--------
Data model location                    FITS key   Value                  Comment
~~~~~~~~~~~~~~~~~~~                    ~~~~~~~~   ~~~~~                  ~~~~~~~
meta.date                            | DATE     = 2018-11-09T13:23:11.221 / Date this file was created (UTC)
meta.exposure.nframes                | NFRAMES  = 1                    / Number of frames coadded in a group
meta.instrument.name                 | INSTRUME = MIRI                 / Instrument used to acquire the data
meta.model_type                      | DATAMODL = GAIN                 / Type of data model
meta.pedigree                        | PEDIGREE = GROUND               / The pedigree of the reference file
meta.reftype                         | REFTYPE  = GAIN                 / Reference file type
meta.subarray.fastaxis               | FASTAXIS = 1                    / Fast readout axis direction
meta.subarray.slowaxis               | SLOWAXIS = 2                    / Slow reado

Note that the newly-created data model contains only the minimum metadata needed to identify it as a
MIRI GAIN reference data. The data model contains convenience functions you can use to define the rest of the metadata. For example, supposing this GAIN model is valid for the MIRIIFULONG detector of the MRS, valid for any detector readout pattern, channels 3 and 4 and any band and a subset can be extracted for any subarray. The following statements will add the missing instrument metadata.

In [4]:
gain_model.set_instrument_metadata( detector='MIRIFULONG', channel='34', band='N/A')
gain_model.set_subarray_metadata( subarray='GENERIC')

There are two alternative ways of defining the exposure. The most vertsatile way is to use the exposure metadata setting convenience function, shown below. The function will create default keywords for parameters, such as NFRAMES and ROWRESETS, which are always the same for MIRI data.

In [5]:
gain_model.set_exposure_metadata( readpatt='N/A', nints=None, ngroups=None )

However, if the only relevant parameter is the detector readout pattern (READPATT), and you don't want any extra keywords created, the READPATT keyword can be defined on its own by directly defining the parameter in the data model hierarchy, like this.

In [14]:
gain_model.meta.exposure.readpatt = 'N/A'

The data model will check that anything written is valid. Try executing the following line to see what happens when you try to set an invalid detector readout mode. This is an example of the data model keeping the data valid.

In [6]:
gain_model.meta.exposure.readpatt = 'NoSuchPattern'

/sw4/smb/anaconda3/envs/miricle.devel/lib/python3.5/site-packages/jwst-0.9.6-py3.5-linux-x86_64.egg/jwst/datamodels/validate.py:32: ValidationWarning: While validating readpatt the following error occurred:
'NoSuchPattern' is not one of ['FAST', 'FASTGRPAVG', 'FASTINTAVG', 'SLOW', 'ANY', 'N/A']

Failed validating 'enum' in schema:
    {'$schema': 'http://stsci.edu/schemas/asdf-schema/0.1.0/asdf-schema',
     'enum': ['FAST', 'FASTGRPAVG', 'FASTINTAVG', 'SLOW', 'ANY', 'N/A'],
     'fits_keyword': 'READPATT',
     'title': 'Readout pattern',
     'type': 'string'}

On instance:
    'NoSuchPattern'
  warnings.warn(errmsg, ValidationWarning)


MIRI Calibration Data Products must also contain housekeeping information, which can be added with the following statements:

In [7]:
gain_model.set_housekeeping_metadata( origin='MIRI European Consortium', author='Joe Bloggs',
                                  version='07.00.00', useafter='2018-11-01',
                                  description='MIRI GAIN data created from a data models example')
gain_model.add_referencefile_history( document='MIRI-TR-0000-UA-Gain', software='IDL script gain.pro',
                                   dataused='MIRI FM data', differences='Metadata update only',
                                   history='This is an extra history record' )

If you know the location of additional metadata within the data model hierarchy, you can set it directly with a
statement like this

In [8]:
gain_model.meta.filename_original = 'MIRI_FM_MIRIFULONG_34_GAIN_04.00.00.fits'

Repeating the metadata query will now reveal the extra metadata.

In [9]:
print( gain_model.info )

MiriGainModel: 
Metadata
--------
Data model location                    FITS key   Value                  Comment
~~~~~~~~~~~~~~~~~~~                    ~~~~~~~~   ~~~~~                  ~~~~~~~
meta.author                          | AUTHOR   = Joe Bloggs           / Author of the reference file
meta.date                            | DATE     = 2018-11-09T13:23:11.221 / Date this file was created (UTC)
meta.description                     | DESCRIP  = MIRI GAIN data created from a data models example / Description of the reference file
meta.exposure.frame_resets           | FRMRSETS = 3                    / Number of extra frame resets for SCA
meta.exposure.groups_averaged        | GRPAVG   = 1                    / Number of groups averaged
meta.exposure.integrations_averaged  | INTAVG   = 1                    / Number of integrations averaged
meta.exposure.nframes                | NFRAMES  = 1                    / Number of frames coadded in a group
meta.exposure.readpatt            

The data contained in the reference file can be displayed by printing the model (which converts its metadata and data into a readable string).

In [10]:
print( gain_model )

MiriGainModel: 
Metadata
--------
Data model location                    FITS key   Value                  Comment
~~~~~~~~~~~~~~~~~~~                    ~~~~~~~~   ~~~~~                  ~~~~~~~
meta.author                          | AUTHOR   = Joe Bloggs           / Author of the reference file
meta.date                            | DATE     = 2018-11-09T13:23:11.221 / Date this file was created (UTC)
meta.description                     | DESCRIP  = MIRI GAIN data created from a data models example / Description of the reference file
meta.exposure.frame_resets           | FRMRSETS = 3                    / Number of extra frame resets for SCA
meta.exposure.groups_averaged        | GRPAVG   = 1                    / Number of groups averaged
meta.exposure.integrations_averaged  | INTAVG   = 1                    / Number of integrations averaged
meta.exposure.nframes                | NFRAMES  = 1                    / Number of frames coadded in a group
meta.exposure.readpatt            

If the data arrays are too large to display sensibly, a summary of the contents can be displayed like this.

In [11]:
print( gain_model.summary )

MiriGainModel: 
One data array:
---------------
Data array 'data' (3 x 3)
  zero=0, non-zero=9; min=0.800000, max=1.300000, mean=1.066667, median=1.100000, std=0.149071



The data model just created can be saved to a FITS file. Remember to use the CDP file naming convention and ensure the version number in the file name matches the version number defined in the metadata. The 'overwrite' parameter ensures any existing file with the same name is overwritten - use this with care.

In [12]:
gain_model.save( "MIRI_FM_MIRIFULONG_34_GAIN_7X.00.00.fits", overwrite=True)

'MIRI_FM_MIRIFULONG_34_GAIN_07.00.00.fits'

Now the data has been saved to a FITS file, the following statement can be used by future code to read the file
and regenerate the data model.

In [13]:
my_data_model = MiriGainModel( "MIRI_FM_MIRIFULONG_34_GAIN_7X.00.00.fits" )
print( my_data_model )

MiriGainModel: 
Metadata
--------
Data model location                    FITS key   Value                  Comment
~~~~~~~~~~~~~~~~~~~                    ~~~~~~~~   ~~~~~                  ~~~~~~~
meta.author                          | AUTHOR   = Joe Bloggs           / Author of the reference file
meta.date                            | DATE     = 2018-11-09T13:23:40.057 / Date this file was created (UTC)
meta.description                     | DESCRIP  = MIRI GAIN data created from a data models example / Description of the reference file
meta.exposure.frame_resets           | FRMRSETS = 3                    / Number of extra frame resets for SCA
meta.exposure.groups_averaged        | GRPAVG   = 1                    / Number of groups averaged
meta.exposure.integrations_averaged  | INTAVG   = 1                    / Number of integrations averaged
meta.exposure.nframes                | NFRAMES  = 1                    / Number of frames coadded in a group
meta.exposure.readpatt            

This example has shown how a data model can created for a reference file containing a simple data array.
A reference file containing a data table can be created in a similar way, except the data tables need to be created by defining lists of tuples (or numpy record arrays), like this example for the MIRI MRS wavelength correction model.

In [14]:
from miri.datamodels.cdp import MiriMrsWavelengthCorrectionModel

optical_data = [('1A', 0.176, 4.87, 5.82, 3320.0, 3710.0),
                ('1B', 0.176, 5.62, 6.73, 3190.0, 3750.0)]
xslice_data =  [(0.2122, 0.3718)]
shift_data =   [(0.000, 0.0, 0.0),
                (0.005, -0.0460, -0.0687),
                (0.010, -0.0924, -0.0687)]

wavcor_model = MiriMrsWavelengthCorrectionModel( wavcorr_optical=optical_data,
                                                 wavcorr_xslice=xslice_data,
                                                 wavcorr_shift=shift_data )
wavcor_model.set_instrument_metadata( detector='MIRIFUSHORT', channel='12', band='N/A')
wavcor_model.set_subarray_metadata( subarray='GENERIC')
wavcor_model.set_housekeeping_metadata( origin='MIRI European Consortium', author='Andy Calibration',
                                  version='07.00.00', useafter='2018-11-01',
                                  description='MRS wavelength correction created from a data models example')
wavcor_model.add_referencefile_history( document='MIRI-TR-0006-ATC', software='IDL script MRS_WavCorr.pro',
                                   dataused='GLAD diffraction modelling', differences='Metadata update only' )

wavcor_model.meta.exposure.readpatt = 'N/A'


Printing the data model reveals its contents.

In [15]:
print( wavcor_model )

MiriMrsWavelengthCorrectionModel: MIRI MRS wavelength correction model
Metadata
--------
Data model location                    FITS key   Value                  Comment
~~~~~~~~~~~~~~~~~~~                    ~~~~~~~~   ~~~~~                  ~~~~~~~
meta.author                          | AUTHOR   = Andy Calibration     / Author of the reference file
meta.date                            | DATE     = 2018-11-09T13:23:52.213 / Date this file was created (UTC)
meta.description                     | DESCRIP  = MRS wavelength correction created from a data models example / Description of the reference file
meta.exposure.nframes                | NFRAMES  = 1                    / Number of frames coadded in a group
meta.exposure.readpatt               | READPATT = N/A                  / Readout pattern
meta.instrument.band                 | BAND     = N/A                  / MIRI sub-channel relevant (MRS)
meta.instrument.ccc_state            | CCCSTATE = OPEN                 / Contamination c

The principles shown here can be extended to the other MIRI data models. The parameters required to create each kind of MIRI data product are described in the MIRI data models reference manual, available here:

http://miri.ster.kuleuven.be/pub/Internal/Software/SoftDevDocs/miri_datamodels.pdf